# Illustration of dipole approximation

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from brainsignals.plotting_convention import (mark_subplots, 
                                              simplify_axes,
                                              cmap_v_e)

np.random.seed(12345)
 
sigma = 0.3
imem = np.array([.3, -.3])
ia = np.abs(imem[0])
l_d = 800.

x = np.array([0., 0.])
y = np.array([0., 0.])
z = np.array([-l_d / 2, l_d / 2])

source_pos = np.array([x, y, z])
dipole_loc = np.array([x.mean(), y.mean(), z.mean()])

# Unit vector pointing from negative to positive current
e_p_vec = np.array([0, 0, -1])
p = ia * l_d * e_p_vec

error_radius = 800

In [ ]:
def two_monopole_potential(elec_locs, source_pos, imem):

    num_elecs = len(elec_locs[0, :])
    num_sources = len(source_pos[0, :])
    r2 = np.zeros((num_elecs, num_sources))
    for elec in range(num_elecs):
        for s in range(num_sources):
            r2[elec, s] = np.sum([(source_pos[n, s] - elec_locs[n, elec])**2
                                 for n in range(3)])

    mapping = 1 / (4 * np.pi * sigma * np.sqrt(r2))
    v_e = mapping @ imem * 1000
    return v_e

def dipole_potential(elec_locs, dipole_pos, p):
    r_ = elec_locs.T - dipole_pos
    V_e = 1000 * 1. / (4 * np.pi * sigma) * (np.dot(r_, p.T)
                    / np.linalg.norm(r_, axis=1) ** 3)
    return V_e

### Calculate two-monopole potential and dipole approximation

In [ ]:
num_elecs = 100

x_0deg = np.zeros(num_elecs)
y_0deg = np.zeros(num_elecs)
z_0deg = np.min(z) - 100 - np.linspace(0, 3000, num_elecs)

x_60deg = np.sin(np.deg2rad(120)) * np.linspace(0, 3000, num_elecs)
y_60deg = np.zeros(num_elecs)
z_60deg = np.cos(np.deg2rad(120)) * np.linspace(0, 3000, num_elecs)

electrode_locs_0deg = np.array([x_0deg, y_0deg, z_0deg])
electrode_locs_60deg = np.array([x_60deg, y_60deg, z_60deg])

lfp_0deg_2m = two_monopole_potential(electrode_locs_0deg, source_pos, imem)
lfp_60deg_2m = two_monopole_potential(electrode_locs_60deg, source_pos, imem)

lfp_0deg_dp = dipole_potential(electrode_locs_0deg, dipole_loc, p)
lfp_60deg_dp = dipole_potential(electrode_locs_60deg, dipole_loc, p)

grid_x, grid_z = np.mgrid[-2000:2001:27, -3000:3002:27]
grid_y = np.zeros(grid_x.shape)

electrode_grid = np.array([grid_x.flatten(),
                           grid_y.flatten(),
                           grid_z.flatten()])

lfp_2m = two_monopole_potential(electrode_grid, source_pos, imem)

lfp_2m = lfp_2m.reshape(grid_x.shape)

lfp_dp_grid = dipole_potential(electrode_grid, dipole_loc, p)
lfp_dp_grid = lfp_dp_grid.reshape(grid_x.shape)

## Plot results

In [ ]:
plt.close("all")
fig = plt.figure(figsize=[6, 2.6])

ax_2m = fig.add_axes([0.0, 0.2, 0.22, 0.7], aspect=1, title="two-monopole",
                        frameon=False, xticks=[], yticks=[],
                        xlim=[-2000, 2000], ylim=[-3100, 3100])
ax_dp = fig.add_axes([0.22, 0.2, 0.22, 0.7], aspect=1, title="dipole",
                        xticks=[], yticks=[], frameon=False,
                        xlim=[-2000, 2000], ylim=[-3100, 3100])
ax_diff = fig.add_axes([0.44, 0.2, 0.22, 0.7], aspect=1, title="difference",
                          xticks=[], yticks=[], frameon=False,
                        xlim=[-800, 800], ylim=[-1200, 1200])

ax2 = fig.add_axes([0.74, 0.57, 0.21, 0.25], xlabel="distance (µm)", ylabel=r"$V_{\rm e}$ (µV)")

ax2b = fig.add_axes([0.74, 0.13, 0.21, 0.25], xlabel="distance in units of $d$",
                       ylabel="relative error", yscale="log",
                       yticks=[1e-4, 1e-3, 1e-2, 1e-1])

num = 15
levels = np.logspace(-3, 0, num=num)

scale_max = 10 #np.max(np.abs(lfp_2m))

levels_norm = scale_max * np.concatenate((-levels[::-1], levels))

colors_from_map = [cmap_v_e(n/(len(levels_norm) - 2))
                   for n in range(len(levels_norm) -1)]

colors_from_map[num - 1] = (1.0, 1.0, 1.0, 1.0)

ep_2m = ax_2m.contourf(grid_x, grid_z, lfp_2m,
                           zorder=2, colors=colors_from_map,
                           levels=levels_norm, extend='both')

ax_2m.contour(grid_x, grid_z, lfp_2m, colors='k',
              linewidths=(1), zorder=2, levels=levels_norm)

ep_dp = ax_dp.contourf(grid_x, grid_z, lfp_dp_grid,
                           zorder=2, colors=colors_from_map,
                           levels=levels_norm, extend='both')

ax_dp.contour(grid_x, grid_z, lfp_dp_grid, colors='k',
              linewidths=(1), zorder=2, levels=levels_norm)

ep_diff = ax_diff.contourf(grid_x, grid_z, lfp_dp_grid - lfp_2m,
                           zorder=2, colors=colors_from_map,
                           levels=levels_norm, extend='both')

ax_diff.contour(grid_x, grid_z, lfp_dp_grid - lfp_2m, colors='k',
                linewidths=(1), zorder=2,
           levels=levels_norm)

imgs = [ep_2m, ep_dp, ep_diff]

dist_0deg = np.sqrt(x_0deg ** 2 + z_0deg ** 2)
dist_60deg = np.sqrt(x_60deg ** 2 + z_60deg ** 2)

idxs_0deg = np.where(dist_0deg > error_radius)
idxs_60deg = np.where(dist_60deg > error_radius)
for i, ax in enumerate([ax_2m, ax_dp, ax_diff]):
    ax.plot(x, z, 'o', c='k')
    [ax.plot(x[i], z[i], '+_'[i], c='w') for i in range(2)]

    if i < 2:
        ax.plot(x_0deg[idxs_0deg], z_0deg[idxs_0deg], ['-', '--'][i], c='b')
        ax.plot(x_60deg[idxs_60deg], z_60deg[idxs_60deg], ['-', '--'][i], c='r')

    ax.add_patch(plt.Circle((0, 0), radius=error_radius,
                               color='none', zorder=50, ls='--',
                               fill=True, ec='cyan', lw=1., clip_on=False))
    ax_x1, ax_y1, ax_w, ax_h = ax.get_position().bounds

    cax = fig.add_axes([ax_x1, 0.19, ax_w, 0.01], frameon=False)
    cbar = fig.colorbar(imgs[i], cax=cax, orientation="horizontal")
    cbar.set_label(r'$V_{\rm e}$ (µV)', labelpad=-5)
    cbar.set_ticks(scale_max * np.array([-1, -0.1, -0.01, 0, 0.01, 0.1, 1]))

    cax.set_xticklabels(cax.get_xticklabels(), rotation=50)

ax2.axvline(error_radius, lw=1, c='cyan', ls='--')
l1, = ax2.plot(dist_0deg[idxs_0deg], lfp_0deg_2m[idxs_0deg], 'b', lw=1)
l2, = ax2.plot(dist_0deg[idxs_0deg], lfp_0deg_dp[idxs_0deg], 'b--', lw=1)
l3, = ax2.plot(dist_60deg[idxs_60deg], lfp_60deg_2m[idxs_60deg], 'r', lw=1)
l4, = ax2.plot(dist_60deg[idxs_60deg], lfp_60deg_dp[idxs_60deg], 'r--', lw=1)

rel_error_0deg = (np.abs(lfp_0deg_2m - lfp_0deg_dp) / np.max(np.abs(
                        lfp_0deg_2m)))[idxs_0deg]
rel_error_60deg = (np.abs(lfp_60deg_2m - lfp_60deg_dp) / np.max(np.abs(
                        lfp_60deg_2m)))[idxs_60deg]

dists_in_ds_0deg = dist_0deg[idxs_0deg] / l_d
dists_in_ds_60deg = dist_60deg[idxs_60deg] / l_d

l0_deg, = ax2b.plot(dists_in_ds_0deg, rel_error_0deg, 'b', lw=1)
l60_deg, = ax2b.plot(dists_in_ds_60deg, rel_error_60deg, 'r', lw=1)

ax2.legend([l1, l2, l3, l4], [r"2-monop., $\theta=180^{\circ}$",
                              r"dipole, $\theta=180^{\circ}$",
                              r"2-monop., $\theta=120^{\circ}$",
                              r"dipole, $\theta=120^{\circ}$",
                              ], frameon=False, ncol=1,
                              loc=(0.1, 0.5))

ax2b.legend([l0_deg, l60_deg], [r"$\theta=180^{\circ}$",
                                r"$\theta=120^{\circ}$"],
            frameon=False, loc=(0.35, 0.55))

ax_2m.plot([1300, 1300], [3000, 2000], lw=1, c='k', clip_on=False)
ax_2m.text(1400, 2500, "1000\nµm", va='center', zorder=1000, clip_on=False)

ax_dp.plot([1300, 1300], [3000, 2000], lw=1, c='k', clip_on=False)
ax_dp.text(1400, 2500, "1000\nµm", va='center', zorder=100000, clip_on=False)

ax_diff.plot([380, 380], [750, 1150], lw=1, c='k', clip_on=False)
ax_diff.text(400, 950, "400\nµm", va='center', zorder=1000)

simplify_axes([ax2, ax2b])
mark_subplots([ax_2m, ax_dp, ax_diff, ax2, ax2b], ypos=1.1, xpos=0.)
plt.savefig("dipole_approximation.pdf")

## Now we want to look at how a two-monopole decays with distance 

In [ ]:
num_elecs = 200

x_0deg = np.zeros(num_elecs)
y_0deg = np.zeros(num_elecs)
z_0deg = -np.linspace(0, 10000, num_elecs)

x_60deg = np.sin(np.deg2rad(120)) * np.linspace(0, 10000, num_elecs)
y_60deg = np.zeros(num_elecs)
z_60deg = np.cos(np.deg2rad(120)) * np.linspace(0, 10000, num_elecs)

x_perp = np.linspace(0, 10000, num_elecs)
y_perp = np.zeros(num_elecs)
z_perp = np.ones(num_elecs) *(-l_d / 2)

elec_locs_0deg = np.array([x_0deg, y_0deg, z_0deg])
elec_locs_60deg = np.array([x_60deg, y_60deg, z_60deg])
elec_locs_perp = np.array([x_perp, y_perp, z_perp])

lfp_0deg = two_monopole_potential(elec_locs_0deg, source_pos, imem)
lfp_60deg = two_monopole_potential(elec_locs_60deg, source_pos, imem)
lfp_perp = two_monopole_potential(elec_locs_perp, source_pos, imem)

grid_x, grid_z = np.mgrid[-2000:2001:27, -3000:3002:27]
grid_y = np.zeros(grid_x.shape)

elec_locs_grid = np.array([grid_x.flatten(),
                           grid_y.flatten(),
                           grid_z.flatten()])

lfp_grid = two_monopole_potential(elec_locs_grid, source_pos, imem)
lfp_grid = lfp_grid.reshape(grid_x.shape)

In [ ]:
plt.close("all")
fig = plt.figure(figsize=[4.2, 2.7])

ax_2m = fig.add_axes([0.01, 0.17, 0.4, 0.8], aspect=1,
                        frameon=False, xticks=[], yticks=[],
                        xlim=[-2000, 2000], ylim=[-3100, 3100])

ax2 = fig.add_axes([0.57, 0.17, 0.4, 0.7], xlabel="distance (mm)",
                      xscale="log", yscale="log",
                      ylabel=r"$V_{\rm e}$ (µV)")

num = 15
levels = np.logspace(-3, 0, num=num)

scale_max = 10 #np.max(np.abs(lfp_2m))

levels_norm = scale_max * np.concatenate((-levels[::-1], levels))

colors_from_map = [cmap_v_e(n/(len(levels_norm) - 2))
                   for n in range(len(levels_norm) -1)]
colors_from_map[num - 1] = (1.0, 1.0, 1.0, 1.0)


ep_2m = ax_2m.contourf(grid_x, grid_z, lfp_grid,
                           zorder=2, colors=colors_from_map,
                           levels=levels_norm, extend='both')

ax_2m.contour(grid_x, grid_z, lfp_grid, colors='k',
              linewidths=(1), zorder=2, levels=levels_norm)

ax_2m.plot(x, z, 'o', c='k')
[ax_2m.plot(x[i], z[i], '+_'[i], c='w') for i in range(2)]

dist_0deg = np.sqrt(x_0deg ** 2 + z_0deg ** 2)
dist_60deg = np.sqrt(x_60deg ** 2 + z_60deg**2)
dist_perp = np.sqrt(x_perp ** 2)

idxs_0deg = np.where(dist_0deg > error_radius)
idxs_60deg = np.where(dist_60deg > error_radius)
idxs_perp = np.where(dist_perp > error_radius)

ax_2m.plot(x_0deg[idxs_0deg], z_0deg[idxs_0deg], '-', c='b', lw=1)
ax_2m.plot(x_60deg[idxs_60deg], z_60deg[idxs_60deg], '-', c='r', lw=1)
ax_2m.plot(x_perp[idxs_perp], z_perp[idxs_perp], '-', c='orange', lw=1)

ax_2m.add_patch(plt.Circle((0, 0), radius=error_radius,
                           color='none', zorder=50, ls='--',
                           fill=True, ec='cyan', lw=1))
ax_2m.plot(0, 0, 'o', c='cyan', ms=4)
ax_x1, ax_y1, ax_w, ax_h = ax_2m.get_position().bounds

cax = fig.add_axes([ax_x1, 0.19, ax_w, 0.01], frameon=False)
cbar = fig.colorbar(ep_2m, cax=cax, orientation="horizontal")
cbar.set_label(r'$V_{\rm e}$ (µV)', labelpad=0)
cbar.set_ticks(scale_max * np.array([-1, -0.1, -0.01, 0, 0.01, 0.1, 1]))

cax.set_xticklabels(cax.get_xticklabels(), rotation=40)

ax2.axvline(error_radius / 1000, lw=1, c='cyan', ls='--')
l1, = ax2.loglog(dist_0deg[idxs_0deg] / 1000, lfp_0deg[idxs_0deg], 'b', lw=1)
l3, = ax2.loglog(dist_60deg[idxs_60deg] / 1000, lfp_60deg[idxs_60deg], 'r', lw=1)
l4, = ax2.loglog(dist_perp[idxs_perp] / 1000, lfp_perp[idxs_perp], 'orange', lw=1)

ax2.legend([l1, l3, l4], [r"$\theta=180^{\circ}$",
                          r"$\theta=120^{\circ}$",
                          r"perpendicular",
                          ],
           frameon=False, loc=(0.25, 0.84))

# Making 1/r**3 markers
r1 = dist_perp[idxs_perp][-1] / 1000
r0 = r1 * 0.5
r = np.linspace(r0, r1, 10)
slope_factor = np.abs(lfp_perp[idxs_perp])[-1] * r1**3
y = slope_factor / r ** 3
ax2.plot(r, y, lw=1.5, c='k')
ax2.text(r0*0.85, y[-1], "1/$r^3$", ha="left")

# Making 1/r**2 markers
r1 = dist_0deg[idxs_0deg][-1] / 1000
r0 = r1 * 0.5
r = np.linspace(r0, r1, 10)
slope_factor = np.abs(lfp_0deg[idxs_0deg])[-1] * r1**2
y = slope_factor / r ** 2
ax2.plot(r, y, lw=1.5, c='k')
ax2.text(r0*1.1, y[0], "1/$r^2$", ha="left")

ax_2m.plot([1800, 1800], [2000, 1000], lw=1, c='k', clip_on=False)
ax_2m.text(1900, 1500, "1 mm", va='center')

ax2.grid(True)

simplify_axes(ax2)
mark_subplots(ax_2m, "B", ypos=0.98, xpos=0.)
mark_subplots(ax2, "C", ypos=1.12, xpos=0.)
plt.savefig("two_monopole_decay.pdf")